In [1]:
import os
import re
import json
import shutil
import re
import logging
from math import radians, cos, sin, asin, sqrt

In [2]:
import pandas as pd
# from tqdm import tqdm
import numpy as np
import math

from sklearn.metrics import mean_squared_error,explained_variance_score
from sklearn.model_selection import KFold

import os
import glob
import warnings
warnings.filterwarnings('ignore')

In [3]:
!pip install geopandas
!pip install descartes
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple


In [4]:
# baseline只用到gps定位数据，即train_gps_path
train_gps_path = 'train0714.csv'
test_data_path = 'R2_ATest0711.csv'
order_data_path = 'loadingevents.csv'
port_data_path = 'port_fixed_0612.csv'

In [5]:
def get_data(data, mode = 'train'):
    assert mode=='train' or mode=='test'
    
    # data['timestamp'] = pd.to_datetime(data['timestamp'], infer_datetime_format=True)
    data['longitude'] = data['longitude'].astype(float)
    data['loadingOrder'] = data['loadingOrder'].astype(str)
    data['latitude'] = data['latitude'].astype(float)
    data['speed'] = data['speed'].astype(float)
    data['direction'] = data['direction'].astype(float)

    data["TRANSPORT_TRACE"] = data['TRANSPORT_TRACE'].astype(str)
    # data["Start"] = data["TRANSPORT_TRACE"].apply(lambda x: x.split("-")[0])
    if mode=='train':
        data['vesselNextportETA'] = pd.to_datetime(data['vesselNextportETA'], infer_datetime_format=True)
        # data["End"] = data["TRANSPORT_TRACE"].apply(lambda x: x.split("-")[-1])
        # data['StartEnd'] = data['Start'].values+'-'+data["End"].values
    elif mode=='test':
        data['temp_timestamp'] = data['timestamp']
        data['onboardDate'] = pd.to_datetime(data['onboardDate'], infer_datetime_format=True)

    print('========== Finish Collecting Data ==========')
    return data

In [6]:
test_data = pd.read_csv('/home/ma-user/work/FuSai/data/'+test_data_path)
course_list = list(sorted(test_data["TRANSPORT_TRACE"].unique()))
test_data["Port_list"] = test_data["TRANSPORT_TRACE"].apply(lambda x: x.split("-"))
start_list = test_data["TRANSPORT_TRACE"].apply(lambda x: x.split("-")[0])
port_lists = np.unique(test_data["Port_list"])
test_data = get_data(test_data, mode='test')

========== Finish Collecting Data ==========


In [7]:
from sklearn.cluster import DBSCAN
DBSCAN(eps=0.5,  # 邻域半径
min_samples=20,    # 最小样本点数，MinPts
metric='euclidean',
metric_params=None,
algorithm='auto', # 'auto','ball_tree','kd_tree','brute',4个可选的参数 寻找最近邻点的算法，例如直接密度可达的点
leaf_size=30, # balltree,cdtree的参数
p=None, # 
n_jobs=1)

DBSCAN(algorithm='auto', eps=0.5, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=20, n_jobs=1, p=None)

In [8]:
port_list = []
for i in range(len(np.unique(test_data['Port_list'].values))):
    port_list.extend(np.unique(test_data['Port_list'].values)[i])

In [9]:
df_port_short = pd.read_csv('/home/ma-user/work/FuSai/data/port_R2ATest.csv')

In [10]:
def geodistance(x,lng1,lat1,lng2,lat2):
    #lng1,lat1,lng2,lat2 = (120.12802999999997,30.28708,115.86572000000001,28.7427)
    lng1, lat1, lng2, lat2 = map(radians, [float(x[lng1]), float(x[lat1]), float(x[lng2]), float(x[lat2])]) # 经纬度转换成弧度
    dlon=lng2-lng1
    dlat=lat2-lat1
    a=sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    distance=2*asin(sqrt(a))*6371*1000 # 地球平均半径，6371km # unit: m
    distance=round(distance/1000,3) # unit: km
    return distance


def get_new_old(ind,size):
    now = np.zeros((len(ind),5),dtype = int)
    now[:,0] = 0
    now[:,2] = ind
    now[:,1] = now[:,2]-1
    now[:,3] = now[:,2]+1
    now[:,4] = size-1
    new = np.min(now[:,3::],axis=1)
    old = np.max(now[:,0:2],axis=1)
    return new,now[:,2],old

def process_velocity(train_df):    
    lon = train_df['longitude'].values
    lat = train_df['latitude'].values
    speed = train_df['speed'].values
    direction = train_df['direction'].values
    ind = np.where((direction == -1)|(speed == 0))
    train_df["time_used"]=pd.to_datetime(train_df["timestamp"])-pd.to_datetime(train_df["timestamp"].iloc[0])
    train_df["time_used"]=train_df["time_used"].dt.total_seconds()
    time = train_df["time_used"].values
    new,now,old = get_new_old(np.arange(train_df.shape[0]),train_df.shape[0])
    londiff = lon[new] - lon[old]
    londiff[londiff > 350] = londiff[londiff > 350] - 360
    londiff[londiff < -350] = londiff[londiff < -350] + 360
    print('londiff.max = ',max(londiff))
    dx = np.cos(lat[now]*np.pi/180)*londiff
    dy = (lat[new] - lat[old])
    train_df['ds'] = 0.5*111.111*(dx**2+dy**2)**0.5
    # print('ind = ',len(ind[0]),', total_ds =',train_df['ds'].sum())
    if (len(ind[0]) >= 0):
        dir_new, dir_now, dir_old = get_new_old(now[ind],train_df.shape[0])
        speed[dir_now] = np.round(3600*111.111*(dx[dir_now]**2+dy[dir_now]**2)**0.5/(time[dir_new]-time[dir_old]),0)
        # print('speed =',speed[dir_now])
        train_df['speed'] = speed
        direction[dir_now] = np.round(np.arctan2(dx[dir_now],dy[dir_now])*18000/np.pi,0)
        direction[direction < 0] = direction[direction < 0] + 36000.
        train_df['direction'] = direction
        
    return train_df,train_df['ds'].sum()


def cluster_stop_ports(train_df):
    train_stop = train_df[(train_df['speed']<=1) & (train_df['direction']>0)]
    train_stop = train_stop.sort_values(by=['timestamp'],ascending=[True])
    index = np.array(train_stop.index)
    x = train_stop[['longitude','latitude']].values
    y_pred = DBSCAN().fit_predict(x)
    train_stop['port'] = y_pred
    if train_stop['port'].min() == -1:
        ind = np.where(train_stop['port'].unique() == -1)[0][0]
        # print('ind where (y_pred == -1) is:',ind)
        ys = np.unique(y_pred)
        train_stop['port'][train_stop['port'].isin(list(ys[(ind+1)::]))] = train_stop['port'][train_stop['port'].isin(list(ys[(ind+1)::]))]+1
        train_stop['port'][train_stop['port'] == -1] = ind
    # print(train_stop['port'].unique())
    return train_stop, index

def generate_port_stops(train_stop):
    ttc = []
    for j in range(len(train_stop['port'].unique())):
        part = train_stop[train_stop['port'] == train_stop['port'].unique()[j]]
        df_port_short['target_longitude'] = part['longitude'].mean()
        df_port_short['target_latitude'] = part['latitude'].mean()
        df_port_short['distance'] = df_port_short.apply(lambda x:geodistance(x,"LONGITUDE","LATITUDE","target_longitude","target_latitude"),axis=1)
        if df_port_short['distance'].min() > 33.3333:
            ttc.append('GHOST'+str(j))
        else:
            ttc.append(df_port_short[df_port_short['distance'] == df_port_short['distance'].min()]['TRANS_NODE_NAME'].unique()[0])
        # print(df_port_short['distance'].min(),'km away from the nearest port:',ttc[-1])
    return ttc

def generate_vesselCurrentport(train_df):
    train_stop, index = cluster_stop_ports(train_df)
    ttc = generate_port_stops(train_stop)
        
    vesselCurrentport = np.nan*np.zeros((train_df.shape[0],),dtype=object)
    cnt = 0
    for k in index:
        vesselCurrentport[k] = ttc[train_stop['port'].values[cnt]]
        cnt += 1
    train_df['vesselCurrentport'] = vesselCurrentport
    if (len(ttc) >= 2) and (len(train_df['vesselCurrentport'].unique())>=3):
        index_min = train_df[train_df['vesselCurrentport'].isin(ttc)].index.min()
        train_df = train_df[train_df.index>=index_min]
        train_df.index = np.arange(len(train_df))
        train_df['time_used'] = train_df['time_used'] - train_df['time_used'].min()
        index_max = train_df[train_df['vesselCurrentport'].isin(ttc)].index.max()
        train_df = train_df[train_df.index<=index_max]
    names_col = ['loadingOrder','timestamp','longitude','latitude','ds','speed','direction','carrierName','vesselMMSI','vesselCurrentport','TRANSPORT_TRACE','time_used']

    return train_df[names_col],ttc

In [11]:
test_data = pd.read_csv('/home/ma-user/work/FuSai/data/'+test_data_path)
course_list = list(sorted(test_data["TRANSPORT_TRACE"].unique()))
test_data = get_data(test_data, mode='test')
test_orders = sorted(test_data['loadingOrder'].unique())

========== Finish Collecting Data ==========


In [12]:
test_data["time_used"]=pd.to_datetime(test_data["timestamp"])-pd.to_datetime(list(test_data["onboardDate"]),utc=True)
test_data["time_used"]=test_data["time_used"].dt.total_seconds()

In [37]:
test_schedule = None

for i in range(len(test_orders)):
    test_df = test_data[test_data['loadingOrder'] == test_orders[i]]
    test_df = test_df.drop_duplicates(keep='first')
    test_df.index = np.arange(len(test_df))
    # 第一步：清洗不正常的速率和方向
    test_df,total_ds = process_velocity(test_df)
    # 第二步: 找到这个运单号停船的行数
    # if (test_df[(test_df['speed']>=2)].shape[0] < test_df.shape[0]):
    #    test_stop, index = cluster_stop_ports(test_df)
    #    test_df,ttc = generate_vesselCurrentport(test_df)
    # else:
    #    test_df['vesselCurrentport'] = np.nan


    transport_trace = test_df['TRANSPORT_TRACE'].unique()[0].split('-')
    for j in range(len(transport_trace)):
        df_port_ttc = df_port_short[df_port_short['TRANS_NODE_NAME']==transport_trace[j]]
        if j == 0:
            begin_longitude = df_port_ttc['LONGITUDE'].values[0]
            begin_latitude = df_port_ttc['LATITUDE'].values[0]
        test_df['port'+str(j)+'_longitude'] = df_port_ttc['LONGITUDE'].values[0]
        test_df['port'+str(j)+'_latitude'] = df_port_ttc['LATITUDE'].values[0]
        test_df['port'+str(j)+'_dist'] = test_df.apply(lambda x:geodistance(x,"longitude","latitude",\
                                                                      'port'+str(j)+'_longitude','port'+str(j)+'_latitude'),axis=1)


    df_schedule = pd.DataFrame(columns = ['loadingOrder',\
                                          'start_time','start_longitude','start_latitude',\
                                          'end_time','end_longitude','end_latitude',\
                                          'begin_longitude','begin_latitude','begin_port',\
                                          'target_longitude','target_latitude','target_port'\
                                          ,'ds','speed',\
                                          'stop_fraction','direction'])
    df_schedule.loc[df_schedule.shape[0]] = [test_orders[i],\
                                             test_df['time_used'].iloc[0],test_df['longitude'].iloc[0],test_df['latitude'].iloc[0],\
                                             test_df['time_used'].iloc[-1],test_df['longitude'].iloc[-1],test_df['latitude'].iloc[-1],\
                                             begin_longitude,begin_latitude,transport_trace[0],\
                                             df_port_ttc['LONGITUDE'].values[0],df_port_ttc['LATITUDE'].values[0],transport_trace[-1],\
                                             test_df['ds'].sum(),test_df['speed'][test_df['speed'] > 1].mean(),\
                            test_df[test_df['speed'] <= 1].shape[0]/test_df.shape[0],test_df['direction'][test_df['speed'] > 1].mean()]
    df_schedule['route_dist'] = df_schedule.apply(lambda x:geodistance(x,"start_longitude","start_latitude",\
                                                                                  "end_longitude","end_latitude"),axis=1)
    if (len(transport_trace) >= 3) and test_df[test_df['speed'] <= 1].shape[0]/test_df.shape[0] <= 0.95:
        dist = 0
        ind = 1
        dist_col = []
        for j in range(1,len(transport_trace)):
            dist_col.append(test_df['port'+str(j)+'_dist'].iloc[-1])
        dist0 = dist_col[0]
        for j in range(1,len(transport_trace)):
            if (test_df['port'+str(j)+'_dist'].iloc[-1] - test_df['port'+str(j)+'_dist'].iloc[-5]) * \
                (test_df['port'+str(j-1)+'_dist'].iloc[-1] - test_df['port'+str(j-1)+'_dist'].iloc[-5]) < 0\
                and dist0 > min(dist_col):
                print('Processing the order =',test_orders[i],',driving from',transport_trace[j-1],'to',transport_trace[j])
                dist0 = test_df['port'+str(j)+'_dist'].iloc[-1]
                dist += dist0
                ind = j
            if j > ind:
                dist += test_df.apply(lambda x:geodistance(x,'port'+str(j-1)+'_longitude',\
                                            'port'+str(j-1)+'_latitude','port'+str(j)+'_longitude','port'+str(j)+'_latitude'),axis=1)
                number = 2*ind - 1
            else:
                number = 2*ind - 2
        df_schedule['dist_to_dest'] = dist
        df_schedule['number'] = number
    else:
        df_schedule['dist_to_dest'] = df_schedule.apply(lambda x:geodistance(x,"end_longitude","end_latitude",\
                                                                              "target_longitude","target_latitude"),axis=1)
        df_schedule['number'] = 1
    if df_schedule['stop_fraction'].values[0] == 1:
        df_schedule['number'] = df_schedule['number'] + 1
    columns = ['loadingOrder','number','start_time','start_longitude','start_latitude','end_time','end_longitude','end_latitude',\
               'begin_longitude','begin_latitude','begin_port','target_longitude','target_latitude','target_port','ds','speed',\
                                          'stop_fraction','direction','route_dist','dist_to_dest']
    df_schedule = df_schedule[columns]
    df_schedule = df_schedule.sort_values(by=['loadingOrder'])
    print('Finish processing the order =',test_orders[i],',i =',i)
    test_schedule = pd.concat([test_schedule,df_schedule])
test_schedule = test_schedule.fillna(value={'speed':0.,'direction':0.,'rho':1.})

londiff.max =  9.33412700000001
Finish processing the order = AC437723355280 ,i = 0
londiff.max =  0.15884499999999946
Finish processing the order = AC860038925693 ,i = 1
londiff.max =  0.013750000000001705
Finish processing the order = AR701843002140 ,i = 2
londiff.max =  0.03245699999999374
Finish processing the order = AT407433613767 ,i = 3
londiff.max =  0.021366999999997915
Finish processing the order = AV639724236766 ,i = 4
londiff.max =  0.11165000000001157
Finish processing the order = BA498843975994 ,i = 5
londiff.max =  0.8282069999999919
Processing the order = BB470439135271 ,driving from CNNBG to MXZLO
Finish processing the order = BB470439135271 ,i = 6
londiff.max =  0.05835000000000434
Finish processing the order = BE790761427541 ,i = 7
londiff.max =  2.3629069999999786
Finish processing the order = BE898272362291 ,i = 8
londiff.max =  2.0922020000000003
Finish processing the order = BE929753510083 ,i = 9
londiff.max =  3.9292530000000028
Finish processing the order = BK8

Finish processing the order = LN662503508371 ,i = 90
londiff.max =  -0.0039840000000097575
Processing the order = LQ213425127748 ,driving from SGSIN to AEJEA
Finish processing the order = LQ213425127748 ,i = 91
londiff.max =  0.0008219999999998784
Finish processing the order = LR291426429726 ,i = 92
londiff.max =  0.0691500000000076
Finish processing the order = LU273033694339 ,i = 93
londiff.max =  4.150500000000008
Finish processing the order = LY233998601535 ,i = 94
londiff.max =  -0.0024999999999977263
Finish processing the order = LY930806925598 ,i = 95
londiff.max =  -0.009500000000000064
Processing the order = LZ390726493707 ,driving from EGSUE to MATNG
Finish processing the order = LZ390726493707 ,i = 96
londiff.max =  0.03786600000002238
Finish processing the order = MF230492968217 ,i = 97
londiff.max =  3.964687999999981
Finish processing the order = MF839953193357 ,i = 98
londiff.max =  13.155057000000014
Finish processing the order = MI815639344195 ,i = 99
londiff.max =  0.

Finish processing the order = VD712620927603 ,i = 178
londiff.max =  -0.003056000000000836
Finish processing the order = VE716598652307 ,i = 179
londiff.max =  0.27111999999999625
Processing the order = VJ323567531982 ,driving from KRPUS to MXZLO
Finish processing the order = VJ323567531982 ,i = 180
londiff.max =  6.1180459999999925
Finish processing the order = VL607740962486 ,i = 181
londiff.max =  0.012477999999987333
Finish processing the order = VR794987570004 ,i = 182
londiff.max =  0.010150000000010095
Finish processing the order = VT466469320985 ,i = 183
londiff.max =  0.01205299999999454
Finish processing the order = VT660611547333 ,i = 184
londiff.max =  0.22043999999999642
Finish processing the order = VY163799060157 ,i = 185
londiff.max =  -0.009766999999999193
Finish processing the order = VY379099044546 ,i = 186
londiff.max =  0.16600999999998578
Finish processing the order = VZ162290304055 ,i = 187
londiff.max =  0.016189999999994598
Finish processing the order = VZ66558

In [39]:
# test_schedule = test_schedule.fillna(value={'speed':0.,'direction':0.,'rho':1.})
test_schedule = test_schedule.sort_values(by = ['loadingOrder'],ascending = [True])
test_schedule.to_csv('/home/ma-user/work/FuSai/data/test_schedule.csv',index=False)

In [38]:
test_schedule

,loadingOrder,number,start_time,start_longitude,start_latitude,end_time,end_longitude,end_latitude,begin_longitude,begin_latitude,begin_port,target_longitude,target_latitude,target_port,ds,speed,stop_fraction,direction,route_dist,dist_to_dest
0,AC437723355280,1,0.0,113.895823,22.383463,905050.0,162.527333,-21.223300,113.863058,22.559462,CNSHK,-71.642993,-33.030843,CLVAP,7233.013003,26.646341,0.017964,12958.070122,7162.936,11684.294
0,AC860038925693,1,0.0,114.290705,22.564435,455150.0,103.092600,1.483327,114.275347,22.577700,CNYTN,-5.812980,35.788207,MATNG,2812.895522,24.868421,0.025641,22206.447368,2637.942,11601.618
0,AR701843002140,1,0.0,113.694225,22.630953,2000.0,113.737135,22.522195,113.863058,22.559462,CNSHK,103.704620,1.303141,SGSIN,12.874376,22.210526,0.000000,15890.000000,12.871,2596.698
0,AT407433613767,1,0.0,104.196312,1.287627,849960.0,43.505220,12.552002,114.139709,22.419915,HKHKG,4.885655,43.430234,FRFOS,7006.465551,28.855263,0.146067,27262.789474,6799.112,5038.529
0,AV639724236766,1,0.0,113.894683,22.451200,17098.0,114.265950,21.757183,113.863058,22.559462,CNSHK,80.282252,13.066265,INMAA,110.163716,22.435714,0.000000,15252.142857,86.130,3723.837
0,BA498843975994,1,0.0,113.899117,22.461683,77686.0,118.285350,24.225433,113.863058,22.559462,CNSHK,130.410386,33.660845,JPHKT,582.093519,25.831818,0.000000,10359.909091,488.827,1576.557
0,BB470439135271,7,0.0,129.496267,34.140347,85815.0,135.931387,38.613280,114.275347,22.577700,CNYTN,-89.829556,13.581498,SVAQJ,760.400176,31.725566,0.000000,4876.803081,760.735,13853.655
0,BE790761427541,1,0.0,113.883267,22.444100,36003.0,113.890616,20.098216,113.863058,22.559462,CNSHK,50.618590,26.135392,BHBAH,310.298333,28.740741,0.000000,15896.543210,260.852,6444.569
0,BE898272362291,1,0.0,114.273142,22.563410,146650.0,120.330858,22.550167,114.275347,22.577700,CNYTN,-104.305571,19.085961,MXZLO,643.798916,31.384615,0.717391,10423.076923,622.017,13311.457
0,BE929753510083,1,0.0,114.285403,22.576393,238450.0,122.399372,30.541828,114.275347,22.577700,CNYTN,-79.882990,9.352609,PAONX,1426.588993,30.215909,0.000000,12477.727273,1197.382,14981.314


In [23]:
test_df[test_df['speed'] <= 1].shape[0]

62

In [25]:
test_df.shape[0]

62